In [ ]:
import os
import numpy as np
import glob
import PIL.Image as Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on GPU.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls /content/gdrive/MyDrive

 1614671043843.gdoc
 31228
 31228_Arunima_Garg_Documents.pdf
 31228_Arunima_Garg.pdf
 31228_Arunima_Internship_Report.docx
 41148_LP3_Assignments
 41245_Maitreyee_NLP_A3.ipynb
 6.cpp.gdoc
'ACM group.jpeg'
 Arunima_Garg_Aadhar_Card.pdf
'Arunima Garg Internship Letter (1).pdf'
'Arunima Garg Internship Letter (2).pdf'
'Arunima Garg Internship Letter (3).pdf'
'Arunima Garg Internship Letter.pdf'
'Arunima Garg Profile_41228.pdf'
'Arunima Garg Profile.pdf'
'Arunima_Garg_resume (1).pdf'
'ArunimaGarg_Resume (1).pdf'
'Arunima_Garg_resume (2).pdf'
'ArunimaGarg_Resume (2).pdf'
'Arunima_Garg_resume (3).pdf'
'ArunimaGarg_Resume (3).pdf'
 Arunima_Garg_resume_final.pdf
 Arunima_Garg_resume.pdf
 ArunimaGarg_Resume.pdf
 ArunimaGarg.zip
'Arunima Photo.jpg'
'Arunima Web Developor.pdf'
 Australia.gslides
 Be_8
 binary_tree.cpp
 C2K20106556_ArunimaGarg_Documents.pdf
'C2K20106556_ArunimaGarg_Resume (1).pdf'
 C2K20106556_ArunimaGarg_resume.pdf
 C2K20106556_ArunimaGarg_Resume.pdf
'CIS-ITSM Certification'
'Cla

In [ ]:
class Hotdog_NotHotdog(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path='/content/gdrive/MyDrive/hotdog_nothotdog'):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path +'/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: id for id, c in enumerate(image_classes)}
        self.image_paths = glob.glob(data_path + '/*/*.jpg')

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        c = os.path.split(os.path.split(image_path)[0])[1]
        y = self.name_to_label[c]
        X = self.transform(image)
        return X, y


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import os, glob

size = 128
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size, scale=(0.8, 1.0)),   # random crop & resize
    transforms.RandomHorizontalFlip(),                      # flip 50% of the time
    transforms.RandomRotation(15),                          # rotate between -15° and +15°
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
])

# ✅ Test data (no augmentation, only resize)
test_transform = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
])

# -------------------
# Datasets & Loaders
# -------------------
trainset = Hotdog_NotHotdog(train=True, transform=train_transform, data_path='/content/gdrive/MyDrive/hotdog_nothotdog')
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=3)

testset = Hotdog_NotHotdog(train=False, transform=test_transform, data_path='/content/gdrive/MyDrive/hotdog_nothotdog')
test_loader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=3)

In [ ]:
# images, labels = next(iter(train_loader))
# plt.figure(figsize=(20,10))

# for i in range(21):
#     plt.subplot(5,7,i+1)
#     plt.imshow(np.swapaxes(np.swapaxes(images[i].numpy(), 0, 2), 0, 1))
#     plt.title(['hotdog', 'not hotdog'][labels[i].item()])
#     plt.axis('off')


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# -----------------
# Model definition
# -----------------
class HotdogCNN(nn.Module):
    def __init__(self):
        super(HotdogCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.6)   # increased dropout
        self.gap = nn.AdaptiveAvgPool2d(1)  # Global Average Pooling

        # Fully connected layers
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.gap(x)                # GAP reduces to (B, 256, 1, 1)
        x = torch.flatten(x, 1)        # flatten to (B, 256)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x




In [ ]:
# -----------------
# Training setup
# -----------------
model = HotdogCNN().to(device)

# ✅ Label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

# ✅ Use adaptive LR scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2
)

num_epochs = 30
patience = 5   # early stopping patience
best_loss = float("inf")
patience_counter = 0

In [ ]:


# -----------------
for epoch in range(num_epochs):
    # ---- Training ----
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    # ---- Evaluation ----
    model.eval()
    test_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss /= len(test_loader)
    test_acc = 100 * correct / total

    # ---- Learning rate update ----
    scheduler.step(test_loss)

    # ---- Print stats ----
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

    # ---- Early stopping ----
    if test_loss < best_loss:
        best_loss = test_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_hotdog_model.pth")  # save best model
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered 🚨")
            break


Epoch [1/30] Train Loss: 0.6322, Train Acc: 66.88% | Test Loss: 0.7715, Test Acc: 48.07%
Epoch [2/30] Train Loss: 0.5642, Train Acc: 74.40% | Test Loss: 0.7279, Test Acc: 63.64%
Epoch [3/30] Train Loss: 0.5422, Train Acc: 76.89% | Test Loss: 0.6280, Test Acc: 68.26%
Epoch [4/30] Train Loss: 0.5293, Train Acc: 77.58% | Test Loss: 0.5303, Test Acc: 77.12%
Epoch [5/30] Train Loss: 0.5004, Train Acc: 80.41% | Test Loss: 0.5417, Test Acc: 77.82%
Epoch [6/30] Train Loss: 0.5024, Train Acc: 80.80% | Test Loss: 0.5237, Test Acc: 77.82%
Epoch [7/30] Train Loss: 0.4988, Train Acc: 80.26% | Test Loss: 0.5250, Test Acc: 77.12%
Epoch [8/30] Train Loss: 0.4891, Train Acc: 81.97% | Test Loss: 0.5185, Test Acc: 77.39%
Epoch [9/30] Train Loss: 0.4958, Train Acc: 80.51% | Test Loss: 0.8341, Test Acc: 60.04%
Epoch [10/30] Train Loss: 0.4955, Train Acc: 80.80% | Test Loss: 0.5031, Test Acc: 79.22%
Epoch [11/30] Train Loss: 0.4819, Train Acc: 82.17% | Test Loss: 0.8913, Test Acc: 52.85%
Epoch [12/30] Train